In [28]:
import requests

def submit_request(text, task, language):
    """Submits a request to the covid disinformation API

    Parameters
    ----------
    text : string
        The text on which the prediction will be performed
    task : string
        Classification task {binary, multiclass}        
    language : string
        Language of the submitted text {arabic, english, bulgarian, dutch} 

    Returns
    -------
    key : str
        key with which the result can later be fetched
    """
    endpoint = "https://webapi.tanbih.org/covid19disformation/%s/" % (language)
    r = requests.post(endpoint, data={"text": text, "task":task})
    response = r.json()
    if response["success"] is True:
        return response["key"]
    else:
        raise Exception("Requestion submission failed")


def fetch_result(key, language):
    """Polls for the response and calls the provided callback when a result is
        available

    Parameters
    ----------
    key : str
        The key from submit_request
    language : string
        Language of the submitted text {english, arabic}

    Returns
    -------
    key : object
        Object with label and probability from the question's classifier
    """
    endpoint = "https://webapi.tanbih.org/covid19disformation/%s/?key=%s" % (language, key)
    r = requests.get(endpoint)
    response = r.json()

    while response["success"] is False:
        if "error" in response and response["error"] == "result is not ready":
            # processing not complete yet, retry
            print("retrying")
            r = requests.get(endpoint)
            response = r.json()
        else:
            raise Exception("Result fetching failed")

    return response["result"]

def predict(text,language,task):
    scores = []
    key = submit_request(text, task, language)
    response = fetch_result(key, language)
    return response


In [29]:
text="COVID19 is going away in a month"
language="english"
task="multiclass"
response = predict(text,language,task)
response

{'Q1': {'label': 'yes',
  'label_dict': [{'label': 'no', 'probability': 0.15678865642301215},
   {'label': 'yes', 'probability': 0.8432113435769877}],
  'probability': 0.8432113435769877},
 'Q2': {'label': '2_no_probably_contains_no_false_info',
  'label_dict': [{'label': '1_no_definitely_contains_no_false_info',
    'probability': 0.050706865064107834},
   {'label': '2_no_probably_contains_no_false_info',
    'probability': 0.5779626575586084},
   {'label': '3_not_sure', 'probability': 0.3291630517726161},
   {'label': '4_yes_probably_contains_false_info',
    'probability': 0.030900012540850164},
   {'label': '5_yes_definitely_contains_false_info',
    'probability': 0.011267413063817797}],
  'probability': 0.5779626575586084},
 'Q3': {'label': '4_yes_probably_of_interest',
  'label_dict': [{'label': '1_no_definitely_not_of_interest',
    'probability': 0.08260910659194237},
   {'label': '2_no_probably_not_of_interest',
    'probability': 0.07219668562310694},
   {'label': '3_not_sur